In [1]:
using ParallelTemperingMonteCarlo

In [11]:
n_atoms = 13

# temperature grid
ti = 5.
tf = 16.
n_traj = 32

temp = TempGrid{n_traj}(ti,tf) 

# MC simulation details
mc_cycles = 30000 #default 20% equilibration cycles on top

mc_sample = 1  #sample every mc_sample MC cycles

#move_atom=AtomMove(n_atoms) #move strategy (here only atom moves, n_atoms per MC cycle)
displ_atom = 0.1 # Angstrom
n_adjust = 100

max_displ_atom = [0.1*sqrt(displ_atom*temp.t_grid[i]) for i in 1:n_traj]

mc_params = MCParams(mc_cycles, n_traj, n_atoms, mc_sample = mc_sample, n_adjust = n_adjust)

#moves - allowed at present: atom, volume and rotation moves (volume,rotation not yet implemented)
move_strat = MoveStrategy(atom_moves = n_atoms)  

#ensemble
ensemble = NVT(n_atoms)

#ELJpotential for neon
#c1=[-10.5097942564988, 0., 989.725135614556, 0., -101383.865938807, 0., 3918846.12841668, 0., -56234083.4334278, 0., 288738837.441765]
#elj_ne1 = ELJPotential{11}(c1)

c=[-10.5097942564988, 989.725135614556, -101383.865938807, 3918846.12841668, -56234083.4334278, 288738837.441765]
pot = ELJPotentialEven{6}(c)

#starting configurations
#icosahedral ground state of Ne13 (from Cambridge cluster database) in Angstrom
pos_ne13 = [[2.825384495892464, 0.928562467914040, 0.505520149314310],
[2.023342172678102,	-2.136126268595355, 0.666071287554958],
[2.033761811732818,	-0.643989413759464, -2.133000349161121],
[0.979777205108572,	2.312002562803556, -1.671909307631893],
[0.962914279874254,	-0.102326586625353, 2.857083360096907],
[0.317957619634043,	2.646768968413408, 1.412132053672896],
[-2.825388342924982, -0.928563755928189, -0.505520471387560],
[-0.317955944853142, -2.646769840660271, -1.412131825293682],
[-0.979776174195320, -2.312003751825495, 1.671909138648006],
[-0.962916072888105, 0.102326392265998,	-2.857083272537599],
[-2.023340541398004, 2.136128558801072,	-0.666071089291685],
[-2.033762834001679, 0.643989905095452, 2.132999911364582],
[0.000002325340981,	0.000000762100600, 0.000000414930733]]

#convert to Bohr
AtoBohr = 1.8897259886
pos_ne13 = pos_ne13 * AtoBohr

length(pos_ne13) == n_atoms || error("number of atoms and positions not the same - check starting config")

#boundary conditions 
#bc_ne13 = SphericalBC(radius=5.32*AtoBohr)   #5.32 Angstrom

bc_ne13 = init_AdjacencyBC(pos_ne13,50.)

#starting configuration
start_config = Config(pos_ne13, bc_ne13)

#histogram information
n_bin = 100
#en_min = -0.006    #might want to update after equilibration run if generated on the fly
#en_max = -0.001    #otherwise will be determined after run as min/max of sampled energies (ham vector)

#construct array of MCState (for each temperature)
mc_states = [MCState(temp.t_grid[i], temp.beta_grid[i], start_config, pot; max_displ=[max_displ_atom[i],0.01,1.]) for i in 1:n_traj]

#results = Output(n_bin, max_displ_vec)
results = Output{Float64}(n_bin; en_min = mc_states[1].en_tot)


Output{Float64}(100, 0.0, 0.0, Float64[], Float64[], Float64[], Vector{Float64}[], Float64[], Float64[], Float64[], Float64[], Float64[])

Testing with the old BC

In [3]:
bc_ne_sph13 = SphericalBC(radius=5.32*AtoBohr)   #5.32 Angstrom

#starting configuration
start_config1 = Config(pos_ne13, bc_ne_sph13)

#histogram information
n_bin = 100
#en_min = -0.006    #might want to update after equilibration run if generated on the fly
#en_max = -0.001    #otherwise will be determined after run as min/max of sampled energies (ham vector)

#construct array of MCState (for each temperature)
mc_states1 = [MCState(temp.t_grid[i], temp.beta_grid[i], start_config1, pot; max_displ=[max_displ_atom[i],0.01,1.]) for i in 1:n_traj]

#results = Output(n_bin, max_displ_vec)
results = Output{Float64}(n_bin; en_min = mc_states1[1].en_tot)

Output{Float64}(100, 0.0, 0.0, Float64[], Float64[], Float64[], Vector{Float64}[], Float64[], Float64[], Float64[], Float64[], Float64[])

In [6]:
#using Pkg; Pkg.add("BenchmarkTools")
using BenchmarkTools

@benchmark atom_move!(mc_states1[1], 2 ,pot,ensemble)

BenchmarkTools.Trial: 10000 samples with 200 evaluations.
 Range (min … max):  399.525 ns …  19.448 μs  ┊ GC (min … max): 0.00% … 96.04%
 Time  (median):     427.480 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   512.836 ns ± 533.031 ns  ┊ GC (mean ± σ):  3.43% ±  3.33%

  █▇▅▄▃▃▃▄▄▄▃▃▂▂▂▁▂▂▂▂▂▂▂▂▂▂    ▁ ▁▁                            ▂
  ███████████████████████████████████████▇▇▇▇▆▆▆▆▆▄▄▅▅▄▄▅▅▄▅▅▄▄ █
  400 ns        Histogram: log(frequency) by time       1.06 μs <

 Memory estimate: 320 bytes, allocs estimate: 2.

In [7]:
@benchmark atom_move!(mc_states[1], 2 ,pot,ensemble)

BenchmarkTools.Trial: 10000 samples with 9 evaluations.
 Range (min … max):  2.430 μs … 964.231 μs  ┊ GC (min … max):  0.00% … 99.44%
 Time  (median):     2.697 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   3.554 μs ±  19.173 μs  ┊ GC (mean ± σ):  11.89% ±  2.21%

  ▆█▇▆▅▄▃▄▄▄▄▄▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁                                 ▂
  ████████████████████████████████████▇▆▇▆▇▇▆▆▆▇▅▆▆▅▄▅▄▄▄▅▄▆▅ █
  2.43 μs      Histogram: log(frequency) by time      7.11 μs <

 Memory estimate: 3.20 KiB, allocs estimate: 53.

In [12]:
@btime ptmc_run!(mc_states, move_strat, mc_params, pot, ensemble, results; save_ham = true)

Total number of moves per MC cycle: 13



equilibration done


MC loop done.
[0.0003353379159949158, 0.00035067306359079704, 0.0003533598480366827, 0.00038372108621373137, 0.0003815210547810432, 0.0004174369993571562, 0.00043349464598757174, 0.0004599283465876704, 0.0004641987346967106, 0.0005310704657648694, 0.0005465017580414445, 0.0006240472262390833, 0.0005658092602767867, 0.00069281800326585, 0.0007410348791969985, 0.0007666938923597615, 0.0009940632953528634, 0.0012927395461337584, 0.0017005311639233796, 0.0015976603060552097, 0.00163265062323615, 0.002512967680625665, 0.0028116565169146283, 0.0028814990746429143, 0.002793027102351234, 0.0024865360487674416, 0.0020393638371166574, 0.002078179044713642, 0.0017409525149549396, 0.0017185401622063295, 0.0014740075496045505, 0.0011679650068474048]
done
Total number of moves per MC cycle: 

13

equilibration done


MC loop done.

...


[-10.884198044620504, -10.429739561295781, -9.99542848772181, -9.57761686610284, -9.179894968465293, -8.789071628651444, -8.415655064282756, -8.0560883887982, -7.705718528948367, -7.361237055858444, -7.02559692694237, -6.719009166131325, -6.425917392173518, -6.124867638385162, -5.8394062247557725, -5.553560168812705, -5.28276248086573, -5.019794158933411, -4.717690845502818, -4.456876900008871, -4.2292943801990095, -3.901851574689044, -3.606102699742419, -3.3823377859106785, -3.1257064131642767, -2.910952569137899, -2.646243209057495, -2.4198318983174443, -2.2562655230006246, -2.110560511130567, -1.959523827546607, -1.8354746618389464]
done
Total number of moves per MC cycle: 

13

equilibration done


MC loop done.


[-21.77004634750821, -20.854599052976113, -19.994328834490968, -19.15701774218842, -18.35700916941299, -17.578440797741845, -16.830086592085973, -16.109912041475077, -15.406897746755055, -14.719884022112629, -14.054030700041492, -13.436160478946013, -12.850879582901129, -12.244621536265635, -11.68084521496406, -11.115016340705873, -10.561881375996544, -10.044460889140353, -9.461061650804083, -8.923322953080568, -8.480372801250828, -7.858387730476073, -7.250786951076533, -6.757977890193058, -6.233985266208912, -5.8165306254449245, -5.30350449976085, -4.8547148975235235, -4.523111636674854, -4.231106480049652, -3.9230822101723635, -3.6872288844094716]
done
  

63.813 s (797655479 allocations: 46.88 GiB)
